In [39]:
#poetryflesh
#stiltambov
#plavanie68
#newtambov
import vk
from datetime import datetime
import os

In [18]:
if __name__ == "__main__":
    token = ""  # Сервисный ключ доступа
    session = vk.Session(access_token=token)  # Авторизация
    vk_api = vk.API(session)

In [19]:
def get_members(groupid):  # Функция формирования базы участников сообщества в виде списка
    first = vk_api.groups.getMembers(group_id=groupid, v=5.124, fields = 'bdate, universities')  # Первое выполнение метода
    data = first["items"]  # Присваиваем переменной первую тысячу id'шников
    count = first["count"] // 1000  # Присваиваем переменной количество тысяч участников
    # С каждым проходом цикла смещение offset увеличивается на тысячу
    # и еще тысяча id'шников добавляется к нашему списку.
    #data = []
    for i in range(1, count+1):
        x = vk_api.groups.getMembers(group_id=groupid, v=5.124, offset = i*1000, fields = 'bdate, universities')["items"]
        #for item in vk_api.users.get(user_ids = x, v = 5.124):
            #data.append(item['response']['id'])
        data = data + x
    return data

In [20]:
poetry = get_members('newtambov')

In [21]:
len(poetry)

6570

In [22]:
def func(data): #убираем закрытые и мертвые страницы
    c = []

    for item in data:
        try:
            if item['is_closed'] == 0:
                c.append(item)
                #c.append(item['id'])
        except TypeError:
            continue
        except KeyError:
            #удаленные страницы
            continue
    return c

In [23]:
filt_data = func(poetry)

In [24]:
len(filt_data)

3890

In [25]:
def get_bdate(bdate): # примет дату и в ответ даст 1 если дата укладывается в ближайшие 3 недели либо 0
    string = ''
    left = datetime.strptime('15.11', '%d.%m') # здесь указываем левую границу вр интервала
    right = datetime.strptime('10.12', '%d.%m')
    c = 0
    for k in bdate:
        if k == '.':
            c += 1
            if c == 2:
                break
        string += str(k)
    date = datetime.strptime(string, '%d.%m')
    #print(date)
    if date>=left and date<=right:
        return 1
    else:
        return 0

def sovp(a,b):
    for i in a:
        for k in b:
            if k == i:
                return 1

In [26]:
# есть ли друзья из вузов
def uni_fr(_user):
    #проверяем есть ли вуз у человека
    try:
        #unis = _user['universities']
        uni_gen = []
        for k in range(len(_user['universities'])):
            uni_gen.append(_user['universities'][k]['name'])
        
        if uni_gen == []:
            return 0
        
        #print(uni_gen)
        
    except KeyError: # если нет, скипаем всю ф-ю
        return 0
    # если все таки есть вуз, начинаем майнить всех друзей из этого вуза
    friends = vk_api.friends.get(user_id = _user['id'], v = 5.124, fields = 'bdate,universities')['items']
    c = 0
    
    for i in range(len(friends)):
        try:
            uni_soft = []
            for l in range(len(friends[i]['universities'])):
                uni_soft.append(friends[i]['universities'][l]['name'])
            if sovp(uni_gen,uni_soft) == 1 and get_bdate(friends[i]['bdate']) == 1:
                c += 1
                #print(friends[i]['id'])
        except KeyError:
            pass
    return c

In [27]:
#функция поиска однофамильцев
def family_survey(_user):
    friends = vk_api.friends.get(user_id = _user['id'], v = 5.124, fields = 'bdate')['items']
    c = 0
    
    general_name = _user['last_name'] #имя юзера
    #print(general_name)
    soft_name = []  # массив под имена его друзей
    # заполним массив   
    c = 0
    for i in range(len(friends)):
        try:
            item = friends[i]['last_name']
            if general_name.startswith(item[:len(item)-2]) and item.startswith(general_name[:len(general_name)-2]) and get_bdate(friends[i]['bdate']) == 1:
                #soft_name.append(friends[i])
                c += 1
        except KeyError:
            pass
    return c
    

In [28]:
def total_friends(_user):
    friends = vk_api.friends.get(user_id = _user['id'], v = 5.124, fields = 'bdate')['items']
    total = 0
    for item in friends:
        try:
            total += 1
        except KeyError:
            pass
    return total

In [29]:
def u_and_f(_user): #выводит количество совпадений по признаку семьи и университета для конкретного пользователя
    total = total_friends(_user)
    c = (family_survey(_user)+uni_fr(_user))*100/total
    return c

In [30]:
arr = []
#for i in range(len(filt_data)):
for i in range(len(filt_data)):
    #print(i)
    if i % 100 == 0:
        print(str(i) + ' из ' + str(len(filt_data)) + '   arr: ' + str(len(arr)))
    try:
        z = u_and_f(filt_data[i])
        if z>=0.23: # интервал рейтинга для каждого пользователя. Чем она выше, тем меньше будет arr[]
            arr.append(filt_data[i]['id'])
            

    except KeyError:
        pass
    except ValueError:
        pass
    except ZeroDivisionError:
        pass
    except VkAPIError:
        pass
print(str(len(filt_data))+ ' из ' +str(len(filt_data)) + '   arr: ' + str(len(arr)))

0 из 3890   arr: 0
100 из 3890   arr: 17
200 из 3890   arr: 30
300 из 3890   arr: 50
400 из 3890   arr: 67
500 из 3890   arr: 83
600 из 3890   arr: 104
700 из 3890   arr: 122
800 из 3890   arr: 139
900 из 3890   arr: 157
1000 из 3890   arr: 175
1100 из 3890   arr: 184
1200 из 3890   arr: 196
1300 из 3890   arr: 208
1400 из 3890   arr: 223
1500 из 3890   arr: 242
1600 из 3890   arr: 255
1700 из 3890   arr: 270
1800 из 3890   arr: 282
1900 из 3890   arr: 292
2000 из 3890   arr: 306
2100 из 3890   arr: 319
2200 из 3890   arr: 330
2300 из 3890   arr: 337
2400 из 3890   arr: 350
2500 из 3890   arr: 360
2600 из 3890   arr: 367
2700 из 3890   arr: 379
2800 из 3890   arr: 390
2900 из 3890   arr: 396
3000 из 3890   arr: 409
3100 из 3890   arr: 415
3200 из 3890   arr: 420
3300 из 3890   arr: 432
3400 из 3890   arr: 439
3500 из 3890   arr: 449
3600 из 3890   arr: 462
3700 из 3890   arr: 469
3800 из 3890   arr: 474
3890 из 3890   arr: 478


In [31]:
for item in arr:
    print(item)

359693
753488
1284610
1443700
1777831
2266257
2893504
3413462
3539748
3590072
4239561
4253183
4266927
4330613
4938885
5060783
5286960
5409748
5456559
6448312
6519146
6912253
7218464
7653163
8186307
8438834
9134680
9170138
9673814
9680647
9799640
10401632
10670231
11112326
11497104
11677421
11920843
11944681
12020635
12158283
12615736
12781544
12791461
13207708
13389694
13441654
13559313
13677402
13895981
14497385
14771823
15050346
15350641
16016059
16137286
16270583
16556671
17207897
17385145
17428836
17459868
18234850
18261507
18348989
18528849
18553345
18626085
18813531
18916117
18919497
19147683
19282154
19302820
19696754
20400588
20773179
20866592
21239958
21639460
21727548
21839953
22010565
22035519
22540847
22599770
22620119
22679983
22690587
22705692
22747362
23292057
23326066
23775100
23947243
24292737
24767185
24769817
25582462
25627869
26068763
26179444
26277388
26289900
26676787
27193168
27207591
27372439
27844785
28082113
28082791
28206423
28943043
28981960
29068118
2909634

In [127]:
def u_and_f_forcheck(_user): #выводит количество совпадений по признаку семьи и университета для конкретного пользователя
    c = family_survey(_user)+uni_fr(_user)
    return c
def check_function(_id):
    us = vk_api.users.get(user_id = _id, v = 5.124, fields = 'city, bdate,universities')
    #print(us[0])
    print('c: ' + str(u_and_f_forcheck(us[0])))

In [128]:
check_function('214105500')

c: 6


In [129]:
for item in arr:
    check_function(item)

c: 1
c: 2
c: 5
c: 1
c: 2
c: 2
c: 1
c: 2
c: 1
c: 1
c: 1
c: 4
c: 3
c: 1
c: 6
c: 1
c: 1
c: 1
c: 4
c: 1
c: 5
c: 15
c: 11
c: 1
c: 1
c: 1
c: 1
c: 1
c: 5
c: 1
c: 4
c: 1
c: 1
c: 1
c: 1
c: 2
c: 1
c: 1
c: 4
c: 1
c: 1
c: 1
c: 12
c: 1
c: 1
c: 5
c: 2
c: 1
c: 1
c: 1
c: 1
c: 1
c: 1
c: 1
c: 6
c: 3
c: 9
c: 1
c: 1
c: 1
c: 1
c: 1
c: 1
c: 1
c: 1
c: 3
c: 1
c: 1
c: 1
c: 4
c: 1
c: 1
c: 1
c: 1
c: 1
c: 1
c: 1
c: 1
c: 1
c: 1
c: 1
c: 7
c: 3
c: 1
c: 19
c: 1
c: 1
c: 1
c: 1
c: 1
c: 1
c: 1
c: 3
c: 2
c: 2
c: 1
c: 1
c: 1
c: 1
c: 8
c: 1
c: 1
c: 1
c: 1
c: 1
c: 1
c: 1
c: 1
c: 1
c: 1
c: 3
c: 1
c: 3
c: 1
c: 1
c: 1
c: 1
c: 1
c: 1
c: 2
c: 13
c: 1
c: 1
c: 1
c: 1
c: 2
c: 8
c: 8
c: 1
c: 1
c: 1
c: 1
c: 5
c: 1
c: 1
c: 1
c: 1
c: 2
c: 1
c: 1
c: 1
c: 1
c: 1
c: 1
c: 11
c: 1
c: 1
c: 3
c: 1
c: 1
c: 1
c: 12
c: 1
c: 1
c: 1
c: 2
c: 1
c: 6
c: 1
c: 1
c: 1
c: 2
c: 1
c: 1
c: 1
c: 2
c: 2
c: 1
c: 13
c: 1
c: 1
c: 1
c: 1
c: 1
c: 2
c: 1
c: 2
c: 2
c: 1
c: 1
c: 1
c: 1
c: 2
c: 1
c: 1
c: 2
c: 1
c: 1
c: 1
c: 1
c: 2
c: 1
c: 1
c: 1
c: 1
c: 9
c: 1
c: 18
c

KeyboardInterrupt: 

In [43]:
len(arr)

478

In [54]:
#поиск в arr инстаграммов в статусах

status_arr = []
for item in arr:
    try:
        arr_user = vk_api.users.get(user_id = item, v = 5.124, fields = 'status')
        status = arr_user[0]['status']
        #print(status)
        if status.find('ig')!=-1 or status.find('inst')!=-1 or status.find('instagram')!=-1 or status.find('@')!=-1 \
        or status.find('_')!=-1:
            
            status_arr.append(str(status))

    except ValueError:
        pass
    except KeyError:
        pass
f = open('ig_status.txt', 'w')
for item in status_arr:
    print(item)
    for i in range(len(item)):
        try:
            f.write(str(item[i]))
        except UnicodeEncodeError:
            pass
    f.write('\n')
f.close()

@mrs.elenaq
Поменьше ворчите, побольше мурчите (=^_^=)
Профессиональный ВЕДУЩИЙ ПРАЗДНИКОВ • Артист , музыкант и юморист) ! 📞viber,WhatSapp 89204849444 inst-sergey_svadba_yarygin
🎼 Студия звукозаписи 👉🏻 https://vk.com/rec_planeta
https://taplink.cc/semeinii_psiholog_tambov	Основатель психологического центра "Мастерская СЧАСТЬЯ"
http://vk.com/app4236781_366329302
Коротко о себе: Противопоказана!... Уж больно до@ера побочных эффектов!😃😇🤫
Marii_a29.03❤
